# Tarea 01
## Nolasco Cabello Adrián Bruce

In [0]:
# Carga de SPARK y creación de sesión
import pyspark
import pyspark.sql.functions as psk 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tarea1').getOrCreate()

In [0]:
datos_crudos = spark.sparkContext.textFile('/FileStore/tables/players_19.csv')
# Eliminamos la cabecera de títulos 


In [0]:
columnas_str = datos_crudos.take(1)[0]
datos_crudos = datos_crudos.filter(lambda renglon : renglon != columnas_str)
columnas = columnas_str.split(',')
columnas

Out[35]: ['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality',
 'club',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'team_position',
 'team_jersey_number',
 'loaned_from',
 'joined',
 'contract_valid_until',
 'nation_position',
 'nation_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'player_traits',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'mo

## Acción 1 de RDD: Extraer IMC máximo por club

In [0]:
def extraer_imc(row):
    row = row.split(',')
    club = str(row[columnas.index('club')])
    height = float(row[columnas.index('height_cm')])/100
    weight = float(row[columnas.index('weight_kg')])
    return (club, weight/(height**2))
rdd_imcs = datos_crudos.map(extraer_imc)

In [0]:
rdd_imcs.take(30)

Out[38]: [('Juventus', 23.735308416025617),
 ('FC Barcelona', 24.913494809688583),
 ('Paris Saint-Germain', 22.20408163265306),
 ('Manchester United', 20.403232301538296),
 ('Manchester City', 21.36686914318855),
 ('Real Madrid', 24.22022684310019),
 ('FC Barcelona', 25.963047941069917),
 ('Real Madrid', 22.30935640886966),
 ('Chelsea', 24.725182932941294),
 ('Atlético Madrid', 24.615210502489816),
 ('Real Madrid', 24.24181207545264),
 ('FC Bayern München', 24.69864962817794),
 ('Atlético Madrid', 22.305470559638533),
 ('Real Madrid', 22.694018931589476),
 ('FC Bayern München', 23.629489603024574),
 ('Juventus', 23.93948099205209),
 ('FC Barcelona', 24.307243558580453),
 ('Real Madrid', 25.50361570247934),
 ('Tottenham Hotspur', 25.18107741059303),
 ('Atlético Madrid', 23.56663223140496),
 ('Chelsea', 24.092970521541954),
 ('Juventus', 24.307243558580453),
 ('Manchester City', 23.38868655818771),
 ('Manchester City', 22.386314277122523),
 ('FC Bayern München', 25.218606946081522),
 ('P

In [0]:
# extraemos el máximo por club y ordenamos
rdd_max_imcs = rdd_imcs.reduceByKey(max).sortBy(lambda renglon: renglon[1], ascending = False)

In [0]:
print("Top 10 clubes por el IMC máximo de sus jugadores")
rdd_max_imcs.take(10)

Top 10 clubes por el IMC máximo de sus jugadores
Out[40]: [('Wycombe Wanderers', 34.71783865673526),
 ('Atlético Nacional', 30.405292525644022),
 ('FC Dallas', 29.761904761904766),
 ('Northampton Town', 29.402920244578837),
 ('Houston Dynamo', 29.33033633946438),
 ('Leicester City', 29.194126488611396),
 ('Peterborough United', 29.060607121599386),
 ('Lincoln City', 28.457139788987625),
 ('Tiburones Rojos de Veracruz', 28.40054796351365),
 ('Al Qadisiyah', 28.39506172839506)]

## Acción II sobre RDD 
## Salario promedio

In [0]:
def extraer_wages(row):
    row = row.split(',')
    club = str(row[columnas.index('club')])
    rep = float(row[columnas.index('wage_eur')])
    return (club, (rep,1.))
rdd_wages = datos_crudos.map(extraer_wages)
rdd_wages.take(2)

Out[58]: [('Juventus', (405000.0, 1.0)), ('FC Barcelona', (565000.0, 1.0))]

In [0]:
# extraemos sumamos los salerios y contamos los jugadores simultaneamente
rdd_inter_wages = rdd_wages.reduceByKey(lambda  x,y: (x[0]+y[0],x[1]+y[1]))
rdd_inter_wages.take(5)

Out[69]: [('Manchester United', (3522000.0, 33.0)),
 ('Manchester City', (3721000.0, 33.0)),
 ('Chelsea', (3266000.0, 33.0)),
 ('Atlético Madrid', (1529000.0, 33.0)),
 ('FC Bayern München', (2413000.0, 28.0))]

In [0]:
print("Top 10 clubes por salario promeido de sus jugadores")
rdd_inter_wages = rdd_inter_wages.map(lambda  x: (x[0],x[1][0]/x[1][1])).sortBy(lambda renglon: renglon[1], ascending = False)
rdd_inter_wages.take(10)

Top 10 clubes por salario promeido de sus jugadores
Out[70]: [('FC Barcelona', 169535.7142857143),
 ('Real Madrid', 150333.33333333334),
 ('Juventus', 130840.0),
 ('Manchester City', 112757.57575757576),
 ('Manchester United', 106727.27272727272),
 ('Chelsea', 98969.69696969698),
 ('Liverpool', 88212.12121212122),
 ('FC Bayern München', 86178.57142857143),
 ('Tottenham Hotspur', 76606.06060606061),
 ('Arsenal', 76181.81818181818)]

## Operaciones con DataFrame

In [0]:
## Carga de DataFrame
df_fifa = spark.read.csv('/FileStore/tables/players_19.csv', header = True, inferSchema = True)

In [0]:
print("Top 5 paises por su jugador más prestigioso")
df_fifa.groupBy('nationality').max('international_reputation').orderBy('max(international_reputation)',ascending=False).show(5)

Top 5 paises por su jugador más prestigiosp
+-----------+-----------------------------+
|nationality|max(international_reputation)|
+-----------+-----------------------------+
|  Argentina|                            5|
|    Germany|                            5|
|     Sweden|                            5|
|    Uruguay|                            5|
|     Brazil|                            5|
+-----------+-----------------------------+
only showing top 5 rows



In [0]:
print("Top 5 clubes con peor habilidad de pase promedio")
df_fifa.groupBy('club').avg('passing').orderBy('avg(passing)',ascending=True).show(5)

Top 5 clubes con peor habilidad de pase promedio
+--------------+------------------+
|          club|      avg(passing)|
+--------------+------------------+
| De Graafschap|              null|
|       Finland|              41.5|
|Bray Wanderers|45.526315789473685|
|         India|              45.9|
|BB Erzurumspor|              46.0|
+--------------+------------------+
only showing top 5 rows



## Consulta SQL

In [0]:
df_fifa.registerTempTable('FIFA19')

In [0]:
query = '''
    SELECT club ,max(wage_eur)- min(wage_eur) as wage_range
    FROM FIFA19
    GROUP BY club
    ORDER BY club
'''
print("Rangos salariales por club")
spark.sql(query).show(5)

Rangos salariales por club
+--------------------+----------+
|                club|wage_range|
+--------------------+----------+
| SSV Jahn Regensburg|      8000|
|1. FC Heidenheim ...|     13000|
|1. FC Kaiserslautern|      4000|
|          1. FC Köln|     25000|
|     1. FC Magdeburg|     16000|
+--------------------+----------+
only showing top 5 rows

